In [ ]:
import numpy as np
import time
import pickle
import json

import math
from random import random, randint, uniform

import os
import shutil

from statistics import median

In [ ]:
def parse_trace(trace_file):
    last_state = {}
    with open(trace_file, "r") as f:
        all_lines = f.readlines()
        final_state_line = all_lines[-2].strip().replace("(", "").replace(")", "").split(" ")
        for item in final_state_line:
            pair = item.split("=")
            last_state[pair[0]] = float(pair[1])
    
    return last_state

In [ ]:
def generate_decoding_domain(observations, radius=10, time_precision=0.005):
    
    pddl_str = "(define (domain flight)\n"
    pddl_str += "\n"
    
    # Predicates
    pddl_str += "(:predicates\n"
    pddl_str += "\t;HS locations\n"
    pddl_str += "\t(flying-straight)\n"
    pddl_str += "\t(adjusting-left)\n"
    pddl_str += "\t(adjusting-right)\n"
    pddl_str += "\n"    
    pddl_str += "\t;monitor locations\n"
    for i in range(len(observations)+1):
        pddl_str += "\t(monitor_{})\n".format(i)
    pddl_str += ")\n"
    pddl_str += "\n"
    
    # Functions
    pddl_str += "(:functions\n"
    pddl_str += "\t;HS variables\n"
    pddl_str += "\t(x1) ; coordinate x\n"
    pddl_str += "\t(x2) ; coordinate y\n"
    pddl_str += "\t(theta) ; orientation\n"
    pddl_str += "\t(v) ; linear speed\n"
    pddl_str += "\t(w) ; angular speed\n"
    pddl_str += "\n"
    
   
    pddl_str += "\t;monitor variables\n"
    pddl_str += "\t(running_time)  ;; time\n"
    pddl_str += "\n"
    
    pddl_str += "\t;objective function\n"
    pddl_str += "\t(max_roughness)\n"
    pddl_str += "\t(roughness)\n"
    pddl_str += "\n"
    
    pddl_str += ")\n"
    pddl_str += "\n"
    
    
    # HS Continuous transitions
    
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += ";;; HS Continuous transitions\n"
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += "\n"
    
    pddl_str += "(:process flow-flying-straight\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(flying-straight)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(increase (x1) (* #t (* (v) (cos (theta)))))\n"
    pddl_str += "\t\t(increase (x2) (* #t (* (v) (sin (theta)))))\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    pddl_str += "(:process flow-adjusting-left\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(adjusting-left)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(increase (x1) (* #t (* (v) (cos (theta)))))\n"
    pddl_str += "\t\t(increase (x2) (* #t (* (v) (sin (theta)))))\n"
    pddl_str += "\t\t(increase (theta) (* #t 0.31415927))\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    pddl_str += "(:process flow-adjusting-right\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(adjusting-right)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(increase (x1) (* #t (* (v) (cos (theta)))))\n"
    pddl_str += "\t\t(increase (x2) (* #t (* (v) (sin (theta)))))\n"
    pddl_str += "\t\t(increase (theta) (* #t -0.31415927))\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    
    
    # HS Discrete transitions
    
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += ";;; HS Discrete transitions\n"
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += "\n"
    
    pddl_str += "(:action stop-adjust-left\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(adjusting-left)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(not (adjusting-left))\n"
    pddl_str += "\t\t(flying-straight)\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    pddl_str += "(:action stop-adjust-right\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(adjusting-right)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(not (adjusting-right))\n"
    pddl_str += "\t\t(flying-straight)\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    pddl_str += "(:action adjust-left\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(flying-straight)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(not (flying-straight))\n"
    pddl_str += "\t\t(adjusting-left)\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    pddl_str += "(:action adjust-right\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(flying-straight)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(not (flying-straight))\n"
    pddl_str += "\t\t(adjusting-right)\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    pddl_str += "(:action toggle-adjust-LR\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(adjusting-left)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(not (adjusting-left))\n"
    pddl_str += "\t\t(adjusting-right)\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    pddl_str += "(:action toggle-adjust-LR\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(adjusting-right)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(not (adjusting-right))\n"
    pddl_str += "\t\t(adjusting-left)\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    
    # HS Invariants
    
    pddl_str += ";;;;;;;;;;;;;;;;;\n"
    pddl_str += ";;; HS Invariants\n"
    pddl_str += ";;;;;;;;;;;;;;;;;\n"
    pddl_str += "\n"
             
    
    # Monitor Continuous transitions
    
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += ";;; Monitor Continuous transitions\n"
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += "\n"
    
    
    for i in range(len(observations)+1):
        pddl_str += "(:process flow-monitor_{}\n".format(i)
        pddl_str += "\t:parameters ()\n"
        pddl_str += "\t:precondition (and\n"
        pddl_str += "\t\t(monitor_{})\n".format(i)
        pddl_str += "\t)\n"
        pddl_str += "\t:effect (and\n"
        pddl_str += "\t\t(increase (running_time) (* #t 1.0))\n"
        pddl_str += "\t)\n"
        pddl_str += ")\n"
        pddl_str += "\n"
        
        
    # Monitor Discrete transitions
    
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += ";;; Monitor Discrete transitions\n"
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += "\n"
        
    for i in range(len(observations)):
        obs = observations[i]
        
        pddl_str += "(:event validate_{}\n".format(i+1)
        pddl_str += "\t:parameters ()\n"
        pddl_str += "\t:precondition (and\n"
        pddl_str += "\t\t(monitor_{})\n".format(i)
        pddl_str += "\t\t(<= {} running_time) ; t_{} - 0.05\n".format(obs[2] - time_precision, i+1)
        pddl_str += "\t\t(> {} running_time) ; t_{} + 0.05\n".format(obs[2] + time_precision, i+1)
        pddl_str += "\t\t(< (+ (^ (- x1 {}) 2) (^ (- x2 {}) 2)) (^ {} 2) ) ; \phi_{}\n".format(round(obs[0],3), round(obs[1],3), radius, i+1)
        pddl_str += "\t)\n"
        pddl_str += "\t:effect (and\n"
        pddl_str += "\t\t(not (monitor_{}))\n".format(i)
        pddl_str += "\t\t(monitor_{})\n".format(i+1)
        pddl_str += "\t\t(increase (roughness) (+ (^ (- x1 {}) 2) (^ (- x2 {}) 2)))\n".format(round(obs[0],3), round(obs[1],3))
        pddl_str += "\t)\n"
        pddl_str += ")\n"
        pddl_str += "\n"
       
             
    # Objective Function    
    
    pddl_str += "(:constraint roughness_bound\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:condition (and\n"
    pddl_str += "\t\t(< roughness max_roughness)\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
        
    pddl_str += ")"
    
    
    return pddl_str
    

In [ ]:
def generate_decoding_problem(observations, theta, v, max_roughness):
    
    pddl_str = "(define (problem problem_name) (:domain platoon)\n"
    pddl_str += "\n"

    # Initial state
    pddl_str += "(:init\n"
    pddl_str += "\t; HS initial state\n"
    pddl_str += "\t(= (x1) 0)\n"
    pddl_str += "\t(= (x2) 0)\n"
    pddl_str += "\t(= (theta) {})\n".format(theta)
    pddl_str += "\t(= (v) {})\n".format(v)
    pddl_str += "\t(flying-straight)\n"
    pddl_str += "\t; monitor initial state\n"
    pddl_str += "\t(= (running_time) 0)\n"
    pddl_str += "\t(monitor_0)\n"
    pddl_str += "\t(= (roughness) 0)\n"
    pddl_str += "\t(= (max_roughness) {})\n".format(max_roughness)
    pddl_str += "\n"
    pddl_str += ")\n"
    
    pddl_str += "(:goal\n"
    pddl_str += "\t(and\n"
    pddl_str += "\t\t(monitor_{})\n".format(len(observations))
    pddl_str += "\t\t(< roughness max_roughness)\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += ")\n"
    
    return pddl_str

In [ ]:
def parse_ENHSP_output(file):
    tags = {
        "Grounding Time" : "grounding_time",
        "Elapsed Time" : "elapsed_time",
        "Planning Time (msec)" : "planning_time",
        "Heuristic Time (msec)" : "heuristic_time",
        "Search Time (msec)" : "search_time",
        "Expanded Nodes" : "expanded",
        "States Evaluated" : "evaluated",
        "Plan-Length" : "plan_length",
        "Problem Solved": "solved",
        "Problem unsolvable": "unsolvable",
        "Delta time heuristic model": "dh",
        "Delta time planning model": "dp",
        "Delta time search-execution model": "de",
        "Delta time validation model": "dv",
        "|F|": "Boolean",
        "|X|": "numeric",
        "|A|": "actions",
        "|P|": "processes",
        "|E|": "events"
    }
    
    json_contents = {"solved": False, "unsolvable": False}
    with open(file,"r") as f:
        for line in f.readlines():           
            tag = line.strip().split(":")[0]
            json_tag = tags.get(tag, None)
            if json_tag is None:
                continue
            elif json_tag == "solved":
                json_contents[json_tag] = True
            elif json_tag == "unsolvable":
                json_contents[json_tag] = True
            else:
                json_contents[json_tag] = float(line.strip().split(":")[1])
                
    return json_contents
                

In [ ]:
num_instances = 10
problems = [i for i in range(num_instances)]
horizons = [2, 4, 6, 8, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
clean_start = True
time_budget = 1800

dfolder = "benchmarks/flight/quality_evolution"
if clean_start:
    if os.path.exists(dfolder):
        shutil.rmtree(dfolder)
    os.makedirs(dfolder)

if clean_start:
    log = open("{}/log".format(dfolder), "w")
else:
    log = open("{}/log".format(dfolder), "a")

with open("flight.pkl", "rb") as f:
    instances = pickle.load(f)
    
for h in horizons:
        
    print("Starting evaluation on size {} problems:\n".format(h))
    log.write("Starting evaluation on size {} problems:\n\n".format(h))
    
    for pnum in problems:
        
        timeout = time_budget
        k = 0
        roughness = 100000000
        
        dname = 'domain_{pnum}_{h}.pddl'.format(pnum=pnum, h=h)      
            
        print("Solving {dname} ...".format(dname=dname))
        log.write("Solving {dname} ...\n".format(dname=dname))
        
        
            
        while True:
            
            pnamek = 'inst_{pnum}_{h}_{k}.pddl'.format(pnum=pnum, h=h, k=k)
            tnamek = 'solution_{pnum}_{h}_{k}'.format(pnum=pnum, h=h, k=k)
            
            domain_str = generate_decoding_domain(instances[pnum]["observations"][:h])           
            with open(dfolder + '/' + dname, 'w') as f:
                f.write(domain_str)
                
            problem_str = generate_decoding_problem(instances[pnum]["observations"][:h], instances[pnum]["init_theta"], instances[pnum]["init_v"], roughness)
            with open(dfolder + '/' + pnamek, 'w') as f:
                f.write(problem_str)
            
            print("Iteration {} (max-roughness = {})".format(k, roughness))
            log.write("Iteration {} (max-roughness = {}): ".format(k, roughness))

            cmd = "timeout {timeout} enhsp_exp -o {dfolder}/{dname} -f {dfolder}/{pnamek} -dp 0.4 -dh 0.4 -de 0.01 -dv 0.01 -s WAStar -h hmrp -pt > out".format(dfolder=dfolder, dname=dname, pnamek=pnamek, pnum=pnum, h=h, timeout=timeout)

            t = time.time()
            os.system(cmd)
            elapsed = time.time() - t

            timeout -= elapsed
            
            json_contents = parse_ENHSP_output("out")
            json_contents["user_time"] = elapsed
            with open(dfolder + "/{tnamek}.json".format(tnamek=tnamek), "w") as json_file:
                json_file.write(json.dumps(json_contents))
                
            if json_contents["solved"]:

                shutil.move(dfolder + "/{pnamek}_search_WAStar_h_hmrp_break_ties_arbitrary.npt".format(pnamek=pnamek), dfolder + "/{tnamek}.npt".format(tnamek=tnamek))
                
                last_state = parse_trace(dfolder + "/{tnamek}.npt".format(tnamek=tnamek))

                roughness = round(last_state["roughness"],3) * 0.9
                log.write("Found solution with roghness = {} using {} seconds\n".format(roughness, elapsed))

                k += 1
            
            else:
                
                print("No solution found")
                log.write("No solution found\n")
                break
        
        print("")
        log.write("\n")
        log.flush()

log.close()